# Colab Starter Code

## Imports

In [0]:
import os
from google.colab import drive

import tensorflow as tf
import numpy as np

from tensorflow.contrib.tensorboard.plugins import projector

!pip install tensorboardcolab
from tensorboardcolab import TensorBoardColab

import cv2
import numpy as np

import tensorflow_hub as hub

import datetime

import matplotlib.pyplot as plt

import time

import pprint

import math

## Functions

In [0]:
def _upload_ssh():
    print("Please upload your SSH key")
    uploaded = files.upload()
    
    ! rm -rf /root/.ssh/*
    ! mkdir /root/.ssh
    ! cp id* /root/.ssh
    ! chmod 400 /root/.ssh
    ! chmod 400 /root/.ssh/*
    !ssh-keyscan bitbucket.org >> /root/.ssh/known_hosts
    
def _mount_drive(drive_path='/content/gdrive/colab_hdd'):
    drive.mount(drive_path, force_remount=True)
    

def _clone_repository(repository_url):
    os.chdir('/content/')
    repository_name = repository_url.split('/')[-1].split('.')[0]
    !rm -rf /content/$repository_name
    !git clone $repository_url

def init_tensorboard():
    global tbc
    timestamp = datetime.datetime.now().strftime('%Y%m%d%H%M%S')
    save_path = '/content/{}/summaries/{}/'.format(_project_name, timestamp)
    tbc = TensorBoardColab(graph_path=save_path)
    global summary_writer
    summary_writer = tf.summary.FileWriter(save_path, tf.get_default_graph())
    
def init_tpu_session():
    if 'COLAB_TPU_ADDR' not in os.environ:
        print('ERROR: Not connected to a TPU runtime. Runtime -> Change runtime type -> Hardware accelerator.')
    else:
        tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
        print ('TPU address is', tpu_address)
        
    global sess
    sess = tf.Session(tpu_address)
    
    devices = sess.list_devices()

    print('TPU devices:')
    pprint.pprint(devices)

### Code for embedding visualization

In [0]:
def visualize_embeddings(embeddings, metadata, experiment_name='default'):
    """Save the embeddings to be visualised using t-sne on TensorBoard
    
    Based on https://medium.com/@vegi/visualizing-higher-dimensional-data-using-t-sne-on-tensorboard-7dbf22682cf2
    
    Metadata example:
        video_index\tframe_index\n
        {}\t{}\n
    """
    tf_embeddings = tf.Variable(np.concatenate(embeddings, 0))

    metadata_path = '/content/{}/embeddings/{}/labels.tsv'.format(_project_name, experiment_name)
    with open(metadata_path, 'w') as metadata_file:
        metadata_file.write(metadata)


    saver = tf.train.Saver([tf_embeddings])
    sess.run(tf_embeddings.initializer)
    saver.save(sess, '/content/{}/embeddings/{}/embeddings.ckpt'.format(_project_name, experiment_name))
    config = projector.ProjectorConfig()

    embedding = config.embeddings.add()

    embedding.tensor_name = tf_embeddings.name
    embedding.metadata_path = metadata_path.split('/')[-1]

    projector.visualize_embeddings(tf.summary.FileWriter('/content/{}/embeddings/{}'.format(_project_name, experiment_name)), config)

In [0]:
def setup(project_name="DefaultProject", clone_repo=True, repo_url="", use_ssh=True, mount_drive=True, drive_folder='colab_hdd', use_tensorboard=True, use_tpu=True):
    global _project_name, _repo_url
    _project_name = project_name
    if clone_repo:
        if use_ssh:
            _upload_ssh()
        _clone_repository(repo_url)
        _repo_url = repo_url
    if mount_drive:
        _mount_drive(drive_folder)
    if use_tensorboard:
        init_tensorboard()
    init_tpu_session()
    global sess
    sess = tf.Session()

# Project

In [0]:
project_name = 'Default' #@param {type:"string"}
clone_repo = False #@param {type:"boolean"}
repo_url = "" #@param {type:"string"}
use_ssh = False #@param {type:"boolean"}
mount_drive = True #@param {type:"boolean"}
drive_folder = 'colab_hdd' #@param {type:"string"}
use_tensorboard = False #@param {type:"boolean"}
use_tpu = True #@param {type:"boolean"}

setup(project_name=project_name, clone_repo=clone_repo, use_ssh=use_ssh, 
      repo_url=repo_url, mount_drive=mount_drive, drive_folder=drive_folder,
      use_tensorboard=use_tensorboard, use_tpu=use_tpu
     )